In [1]:
# checkpoint_path = './train_log/train/model-15'
checkpoint_path = './train_log/train/checkpoint'

In [2]:
from tensorpack.predict.config import PredictConfig
from tensorpack.tfutils import SmartInit

ImportError: cannot import name 'SmartInit'

In [ ]:
from VAE.Model import ModelDesc, Trainer  # , RandomZData
from VAE import get_default_hparams
hps = get_default_hparams()
M = ModelDesc(hps)

In [ ]:
pred_config = PredictConfig(
        session_init=SmartInit(checkpoint_path),
        model=M,
        input_names=['x'],
        output_names=['predict/y_pred']
)

In [ ]:
from tensorpack.predict import OfflinePredictor
predictor = OfflinePredictor(pred_config)

In [7]:
import pymongo
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [8]:
mongo_client = MongoClient('localhost', 27017)
db = mongo_client.crypto_currency
collection = db['ohlcv']

In [9]:
market = 'binance'
symbol = 'BNB/BTC'
timewindow = '1h'
query = { 'market': market, 'symbol': symbol, 'timewindow':timewindow }

df_data = pd.DataFrame(list(collection.find(query)))

In [10]:
cols = ['Open', 'High', 'Low', 'Close', 'Volume']
df_test = df_data[cols].iloc[-hps.M * hps.T:].values
df_test = np.reshape(df_test, newshape=[hps.M, hps.T, hps.D])
print(df_test.shape)

(100, 30, 5)


In [11]:
# tf.TensorSpec((None, self.hps.T, self.hps.D), tf.float32, 'x'),
outputs = predictor(df_test)

In [12]:
outputs[0][-1]

array([[ 0.22112767, -0.18026996]], dtype=float32)

In [13]:
def softmax(x):
    "x shape is [N, Tau, C] when Tau == 1"
    assert np.shape(x)[1] == 1, "Tau is not 1"
    x = np.squeeze(x, axis=(1,))
    e_x = np.exp(x - np.max(x, axis = -1, keepdims = True))
    return e_x / e_x.sum(axis = -1, keepdims = True)

print(softmax(outputs[0])[-10:])

[[0.6558957  0.3441043 ]
 [0.6361076  0.36389244]
 [0.6398929  0.36010712]
 [0.6199106  0.38008943]
 [0.65138793 0.348612  ]
 [0.6009975  0.39900252]
 [0.6401808  0.35981917]
 [0.6554924  0.34450755]
 [0.61280787 0.3871921 ]
 [0.59902346 0.4009766 ]]
